In [1]:
import re
import requests
import warnings
from collections import namedtuple, defaultdict, OrderedDict
import logging

import multiprocessing as mp
import numpy as np
import pandas as pd
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('games.log', mode='a')
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

## Обучим несколько моделей 

In [3]:
%%time

file_path = 'us-financial-news-articles/us-financial-news-articles.txt'

model_skipgram = fasttext.train_unsupervised(file_path, model='skipgram', dim=5)
model_cbow = fasttext.train_unsupervised(file_path, model='cbow', dim=16)
model_skipgram2 = fasttext.train_unsupervised(file_path, model='skipgram', dim=10)

CPU times: user 41min 29s, sys: 14.3 s, total: 41min 43s
Wall time: 4min 17s


In [4]:
!wc -w {file_path}

 20329509 us-financial-news-articles/us-financial-news-articles.txt


In [5]:
!head -1 {file_path}

(Updates prices, adds Trump comments) By Rodrigo Campos NEW YORK, Jan 25 (Reuters) - Mexico's peso seesawed against the dollar on Thursday as U.S. officials sent mixed signals on the greenback, while Argentina's Merval stock index broke the 35,000-point mark for the first time. Several emerging currencies hit multi-year highs against the greenback, with the dollar index languishing at more than three-year lows after U.S. Treasury Secretary Steven Mnuchin departed from traditional U.S. currency policy, saying "obviously a weaker dollar is good for us." The Mexican peso appreciated by more than 1 percent to 18.3025 earlier in the day before U.S. President Donald Trump said Mnuchin had been misinterpreted and that he ultimately wanted the dollar to be strong. Trump's comments helped the dollar to pare losses against major currencies, and the Mexican peso reversed its gains, closing down almost 0.6 percent against the greenback. Elsewhere, Colombia's peso added to Wednesday's 1.48 percent 

In [6]:
len(model_cbow.words)

135642

In [7]:
model_cbow['song']

array([ 4.881617  ,  3.679234  ,  3.7162063 , -0.229449  ,  0.89953285,
        4.7503014 , -3.3211408 ,  3.011697  ,  3.822744  ,  2.224185  ,
        0.3218044 ,  3.3189864 , -2.5510416 ,  4.9163985 ,  5.419481  ,
       -5.750706  ], dtype=float32)

In [8]:
!mkdir models
model_skipgram.save_model('models/skipgram.model')
model_skipgram2.save_model('models/skipgram2.model')
model_cbow.save_model('models/cbow.model')

mkdir: models: File exists


In [ ]:
!ls -lh models

## Реализации классов для игроков

In [10]:
class AbstractPlayer:
    def __init__(self):
        raise NotImplementedError()

    def explain(self, word, n_words):
        raise NotImplementedError()
        
    def guess(self, words, n_words):
        raise NotImplementedError()


if hasattr(model_skipgram2, 'get_nearest_neighbors'):
    print('using latest fasttext version')
    
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model

        def find_words_for_sentence(self, sentence, n_closest):
            neighbours = self.model.get_nearest_neighbors(sentence)
            words = [word for similariry, word in neighbours][:n_closest]
            return words

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)
else:
    print('using older fasttext version')
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model
            self.words = model.get_words()
            self.matrix = np.concatenate([model[word].reshape(1, -1) for word in self.words], axis=0)

        def find_words_for_vector(self, vector, n_closest):
            sims = cosine_similarity(vector.reshape(1, -1), self.matrix).ravel()
            word_sims = pd.Series(sims, index=self.model.get_words()).sort_values(ascending=False)
            return list(word_sims.head(n_closest).index)

        def find_words_for_sentence(self, sentence, n_closest):
            vector = self.model.get_sentence_vector(sentence)
            return self.find_words_for_vector(vector, n_closest)

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)


class RemotePlayer(AbstractPlayer):
    def __init__(self, url, timeout=1):
        self.url = url
        self.timeout = 1
        self.ping()
        
    def ping(self):
        try:
            response = requests.get(self.url)
            assert response.status_code == 200
        except Exception as exc:
            logger.warn(exc)
        
    def explain(self, word, n_words):
        try:
            response = requests.get(
                self.url + '/explain',
                {'word': word, 'n_words': n_words},
                timeout=self.timeout
            )
            word_list = response.json()
        except Exception as exc:
            logger.warning(exc)
            word_list = []
        return word_list
    
    def guess(self, words, n_words):
        try:
            response = requests.get(
                self.url + '/guess',
                {'words': words, 'n_words': n_words},
                timeout=self.timeout
            )
            word_list = response.json()
        except Exception as exc:
            logger.warning(exc)
            word_list = []
        return word_list

using latest fasttext version


In [11]:
# check locally deployed service
# remote_player = RemotePlayer('http://127.0.0.1:5000')

# check remotely deployed service
remote_player = RemotePlayer('https://obscure-everglades-02893.herokuapp.com')

print(remote_player.explain('work', 10))
print(remote_player.guess(['job', 'employee', 'office'], 5))

['work', 'discontent', 'probably:', 'lopid', 'gives', 'putty', 'refund', 'strangest', 'enuff', 'inovative']
['bars;', 'earnings', 'appellate', 'discoverd', 'phage']


In [12]:
local_player = LocalFasttextPlayer(model_skipgram)
print(local_player.explain('work', 10))
print(local_player.guess(['job', 'employee', 'office'], 5))

['pay,”', 'response."', 'favors.', 'devoted', 'ancestry', 'educated', 'choose', 'inserting', 'hierarchical', 'disjointed']
['"value"', 'timeshare', 'Healthineers,', 'Railways,', 'Postbank']


## Игра

In [13]:
class Game:
    def __init__(
        self, players, words, criteria,
        n_rounds, n_explain_words, n_guessing_words,
        random_state=None
    ):
        assert len(players) >= 2
        assert criteria in ('hard', 'soft')
        self.players = players
        self.words = words
        self.criteria = criteria
        self.n_rounds = n_rounds
        self.n_explain_words = n_explain_words
        self.n_guessing_words = n_guessing_words
        
    def remove_repeated_words(self, words):
        unique_words = []
        for c in words:
            if not c in unique_words:
                unique_words.append(c)
        return unique_words
        
    def score_players(self, explainer_name, last_rounds):
        rewards = {
            player: self.n_explain_words + 1 - nround
            for player, nround in last_rounds.items()
        }
        rewards[explainer_name] = sum(rewards.values())
        return rewards

    def create_word_list(self, player, word, n_words):
        reported_words = player.explain(word, n_words)
        explain_words = reported_words[:]
        if self.criteria == 'hard':
            explain_words = explain_words[:n_words]
        explain_words = [re.sub(r'[^\w]', '', c) for c in explain_words]
        explain_words = [c for c in explain_words if word not in c]
        explain_words = self.remove_repeated_words(explain_words)
        if self.criteria == 'soft':
            explain_words = explain_words[:n_words]
        return reported_words, explain_words
    
    def check_criteria(self, word, guessed_words):
        if self.criteria == 'soft':
            guessed = any([word in c for c in guessed_words])
        else:
            guessed = word in guessed_words
        return guessed

    @staticmethod
    def ask_player(player, question, word, n_words):
        method = getattr(player, question)
        return method(word, n_words)
    
    def ask_guessing_players(self, guessing_players, sentence):
        players_guesses = {}
        remote_guessing_players = []
        for player in guessing_players:
            # print(player.api)
            if isinstance(player.api, RemotePlayer):
                remote_guessing_players.append(player)
            else:
                players_guesses[player.name] = player.api.guess(sentence, self.n_guessing_words)

        # print(remote_guessing_players)

        if len(remote_guessing_players) > 0:
            n_processes = np.fmin(len(remote_guessing_players), np.fmax(1, mp.cpu_count() - 1))
            # print(len(remote_guessing_players), np.fmax(1, mp.cpu_count() - 1))
            pool = mp.Pool(n_processes)
            remote_players_guesses = pool.starmap(
                self.ask_player,
                [(p.api, 'guess', sentence, self.n_guessing_words) for p in remote_guessing_players]
            )
            pool.close()

            players_guesses.update(
                zip([player.name for player in remote_guessing_players], remote_players_guesses)
            )
        return players_guesses

    def play_round(self, explaining_player, guessing_players, word, sentence):
        game_round = OrderedDict()
        results = {}
        logger.info(f"HOST: {sentence}")
        game_round.update({f'Explanation for "{word}" ({explaining_player.name})': sentence})

        if False:
            for player in guessing_players:
                guessed_words = player.api.guess(sentence, self.n_guessing_words)
                guessed = self.check_criteria(word, guessed_words)
                results[player.name] = guessed
                logger.info(f'GUESSING PLAYER ({player.name}) to HOST: {guessed_words}')
                logger.info(f'HOST: {guessed}')
                game_round.update({f'Guess ({player.name})': guessed_words})
        else:
            players_guesses = self.ask_guessing_players(guessing_players, sentence)

            for player in guessing_players:
                guessed_words = players_guesses[player.name]
                guessed = self.check_criteria(word, guessed_words)
                results[player.name] = guessed
                logger.info(f'GUESSING PLAYER ({player.name}) to HOST: {guessed_words}')
                logger.info(f'HOST: {guessed}')
                game_round.update({f'Guess ({player.name})': guessed_words})

        return game_round, results

    def play(self, explaining_player, guessing_players, word, criteria):

        logger.info(f'HOST to EXPLAINING PLAYER: the word is "{word}"')

        reported_words, guessing_by = self.create_word_list(explaining_player.api, word, self.n_explain_words)
        logger.info(f'EXPLAINING PLAYER to HOST: my wordlist is {reported_words}')
        logger.info(f'HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is {guessing_by}')

        df = []
        success_rounds = {}
        for iround in range(1, len(guessing_by) + 1):
            if len(guessing_players) == 0:
                break
            logger.info(f'\n===ROUND {iround}===\n')
            game_round, results_round = self.play_round(
                explaining_player=explaining_player,
                guessing_players=guessing_players,
                word=word,
                sentence=guessing_by[:iround],
            )
            for player in guessing_players[:]:
                if (player.name not in success_rounds) and results_round.get(player.name, False):
                    success_rounds[player.name] = iround
                    guessing_players = [p for p in guessing_players if p != player]
            df.append(game_round)
        
        df = pd.DataFrame(df)
        scores = self.score_players(explaining_player.name, success_rounds)
        
        return df, scores
    
    def get_words(self, complete):
        if not complete:
            words = self.words[:]
            np.random.seed(self.random_state)
            np.random.shuffle(words)
        else:
            words = []
            for word in self.words:
                words.extend([word] * len(self.players))
        return words
    
    def get_n_rounds(self, complete):
        if complete:
            return len(self.words)
        else:
            return self.n_rounds
        
    @staticmethod
    def set_console_logging_level(verbose):
        if verbose == 'print_logs':
            console_logging_level = logging.INFO
        else:
            console_logging_level = logging.WARNING
        c_handler.setLevel(console_logging_level)

    def run(self, verbose=False, complete=False):
        self.set_console_logging_level(verbose)
        
        self.run_words = self.get_words(complete=complete)
        self.run_rounds = self.get_n_rounds(complete=complete)
            
        igame = 0
        scores = []
        scores_status = defaultdict(int)
        for r in range(self.run_rounds):
            for explaining_player in self.players:
                guessing_players = [p for p in self.players if p != explaining_player]
                word = self.run_words[igame]
                df, score = self.play(
                    explaining_player, guessing_players, word,
                    criteria=self.criteria
                )
                scores.append(score)
                scores_status[(explaining_player.name, 'explaining')] += score.get(explaining_player.name, 0)
                for player in guessing_players:
                    scores_status[(player.name, 'guessing')] += score.get(player.name, 0)
                igame += 1
                logger.info(f'\n\nSCORES: {score}')
                if verbose:
                    display(df)

        self.scores = pd.DataFrame(scores).fillna(0)
        self.scores.index.name = 'game'
        
        self.scores_status = pd.Series(scores_status).unstack()        

    def report_results(self, each_game=False):
        if each_game:
            print('=== Team scores in each game ===')
            display(self.scores)
        logger.debug(self.scores)
        
        print('=== Team scores, summary ===')
        self.summary = self.scores_status
        self.summary['total'] = self.scores.sum(axis=0)
        self.summary.sort_values('total', ascending=False)
        display(self.summary)
        logger.debug(self.summary)


player = namedtuple('Player', ['name', 'api'])

N_EXPLAIN_WORDS = 10
N_GUESSING_WORDS = 5
N_ROUNDS = 1
CRITERIA = 'soft'

PLAYERS = [
    player('HerokuOrg team', RemotePlayer('https://obscure-everglades-02893.herokuapp.com')),
    player('GKEOrg team', RemotePlayer('http://104.154.210.36')),
    player('skipgram team', LocalFasttextPlayer(model_skipgram)),
    player('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    player('cbow team', LocalFasttextPlayer(model_cbow))
]

# WORDS = [
#     'play', 'master', 'word', 'cocoa', 'coffee',
#     'september', 'jungle', 'spell', 'python', 'world',
#     'cat', 'joy', 'sadness', 'small', 'stick',
# ]
WORDS = ['september']

game = Game(PLAYERS, WORDS, CRITERIA, N_ROUNDS, N_EXPLAIN_WORDS, N_GUESSING_WORDS)
game.run(verbose='print_logs', complete=True)

HOST to EXPLAINING PLAYER: the word is "september"
EXPLAINING PLAYER to HOST: my wordlist is ['september', '24th', 'tatars', 'october', 'november', 'august', '1972', 'american-occupied', '1988-89', '1947']
HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is ['24th', 'tatars', 'october', 'november', 'august', '1972', 'americanoccupied', '198889', '1947']

===ROUND 1===

HOST: ['24th']
GUESSING PLAYER (GKEOrg team) to HOST: ['24th', 'Sorkin’s', 'Amputee', '68th', 'occasion,']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['Simona', 'won', 'Croatia’s', '38-7', 'Dzumhur,']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ["Everton's", 'Kitzbuehel,', 'Kitzbuehel', 'WTA', 'Championship.']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['26th', '29th', '13th', '14th', '106th']
HOST: False

===ROUND 2===

HOST: ['24th', 'tatars']
GUESSING PLAYER (GKEOrg team) to HOST: ['24th', '(AEM-R)', 'Laureus', 'team-leading', '3-2.']
HOST: False
GUESSING PLAYER (skipgram team

,"Explanation for ""september"" (HerokuOrg team)",Guess (GKEOrg team),Guess (skipgram team),Guess (skipgram2 team),Guess (cbow team)
0,[24th],"[24th, Sorkin’s, Amputee, 68th, occasion,]","[Simona, won, Croatia’s, 38-7, Dzumhur,]","[Everton's, Kitzbuehel,, Kitzbuehel, WTA, Cham...","[26th, 29th, 13th, 14th, 106th]"
1,"[24th, tatars]","[24th, (AEM-R), Laureus, team-leading, 3-2.]","[Bellamy’s, RDIF, 2006, show:, Helicopters]","[Ancient, Rescue, Policemen, Rescuers, Seventeen]","[Strikers, bbls, Fighters, Vats, jars]"
2,"[24th, tatars, october]","[24th, pre-tournament, mouths, Isfahan., Assas...","[Jews’, pariah, couples., Mecca’s, bulletins]","[festivities, Barents, Jamaica,, Embobut, serv...","[2012,, 2015, 2015:, 2015;, 2013;]"
3,"[24th, tatars, october, november]","[Isfahan., anti-Barros, “Glee,”, Miseducation,...","[easyJet,, 13-year, bids, oasis, Taiwanese]","[Liberia, Israel-Gaza, Romania,, Valenzuela, V...","[2013;, 2013:, mid-October, 2015;, 2011-14]"
4,"[24th, tatars, october, november, august]","[REUTERS/Erik, anti-Barros, $71.28., “Glee,”, ...","[has..., Taiwanese, bidding, Iceland, open.]","[Liberia, census, Vladivostok, four-day-old, a...","[mid-October, October’s, mid-November, Septemb..."
5,"[24th, tatars, october, november, august, 1972]","[“Jersey, “bookkeeper, “Glee,”, $71.28., Kloten,]","[December,”, Glencore., Dumping, snags, Street.]","[1800s, Vladivostok, Liberia, Kaohsiung, Italy.]","[2011-14, 2013, September,, 2013-14, 2013/14]"
6,"[24th, tatars, october, november, august, 1972...","[“bookkeeper, Isfahan., “Glee,”, anti-Barros, ...","[Bompard, Carrefour's, Cabral, 1800s, hilltop]","[Valenzuela, 97,, Israel-Gaza, Timbuktu, Liberia]","[wounding, uprisings, torturing, seven-man, se..."
7,"[24th, tatars, october, november, august, 1972...","[“Glee,”, “bookkeeper, Isfahan., Mauresmo, wed.]","[pay-television, Goldman's, eleventh, 9mobile,...","[97,, Valenzuela, extinction, Xgeva, Ravenstho...","[uprisings, abduction, seven-man, 2011-14, thr..."
8,"[24th, tatars, october, november, august, 1972...","[Pails, Hill”, “Glee,”, Miseducation, Amelie]","[Goldman's, pay-television, eleventh, 9mobile,...","[97,, Valenzuela, extinction, 1800s, Xgeva]","[uprisings, abduction, seven-man, 2011-14, thr..."


HOST to EXPLAINING PLAYER: the word is "september"
EXPLAINING PLAYER to HOST: my wordlist is ['November', 'October', 'September', 'September.', 'Souvant/Pool/File', 'REUTERS/Guillaume', 'September,', 'Wermuth/File', "September's", 'EMBR3.SA']
HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is ['November', 'October', 'September', 'SouvantPoolFile', 'REUTERSGuillaume', 'WermuthFile', 'Septembers', 'EMBR3SA']

===ROUND 1===

HOST: ['November']
GUESSING PLAYER (HerokuOrg team) to HOST: ['inmates', 'germany', 'american-occupied', 'coast', 'camber']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ["Reserve's", 'Mongolia', '[EIA/S]', 'Bahrain’s', 'October']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['October', '2015.', 'August', 'July', 'Oct.']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['October', 'November/December', 'January-November', 'April-December', 'July-September']
HOST: False

===ROUND 2===

HOST: ['November', 'October']
GUESSING PLAYER (Herok

,"Explanation for ""september"" (GKEOrg team)",Guess (HerokuOrg team),Guess (skipgram team),Guess (skipgram2 team),Guess (cbow team)
0,[November],"[inmates, germany, american-occupied, coast, c...","[Reserve's, Mongolia, [EIA/S], Bahrain’s, Octo...","[October, 2015., August, July, Oct.]","[October, November/December, January-November,..."
1,"[November, October]","[inmates, canada, airfield, germany, italy]","[bourse, Ambatovy, ECB's, mid-October, Baltic]","[mid-October, barreled, June., 28., 2014]","[November, November/December, mid-October, Oct..."
2,"[November, October, September]","[inmates, camber, arbour, march, november]","[[EIA/M], mid-March, Autos, Economists, settle...","[mid-March, mid-November, nine-month, October'...","[November/December, November, April-December, ..."
3,"[November, October, September, SouvantPoolFile]","[frances, candidate:, zarmooni, promo, newspap...","[poll:, (CNB), No.2, Optimism, local):]","[2017-March, census, three-month-long, June., ...","[November, November/December, April-December, ..."
4,"[November, October, September, SouvantPoolFile...","[promo, colossal, boise, newsday, trojan]","[heavyweights, 2013;, s, Births, S&P's]","[85., 28., Nampo, 36., 2014;]","[March;, 2015-16,, 2014/15,, KC-390,, October]"
5,"[November, October, September, SouvantPoolFile...","[weekday, newspot, troost, hollywood, |after]","[Sainsbury's, Sanpaolo., Tesco,, (After, Skagit]","[Peter/File, REUTERS/Staff/Remote, Plunkett/Fi...","[March;, 2014/15,, 2015-16,, KC-390,, Augusto]"
6,"[November, October, September, SouvantPoolFile...","[hollywood, newspot, frances, silvered, afro-f...","[(After, Births, laggards, Wednesday:, time:]","[85., 2017-March, 28., 101., REUTERS/Staff/Rem...","[2015-16,, 2014/15,, October, November, 2014-09,]"
7,"[November, October, September, SouvantPoolFile...","[boise, colossal, cedar, kcal, college:]","[(After, Births, time:, laggards, Copper,]","[85., 101., 2017-March, REUTERS/Staff/Remote, ...","[2015-16,, 2014/15,, October, Augusto, March;]"


HOST to EXPLAINING PLAYER: the word is "september"
EXPLAINING PLAYER to HOST: my wordlist is ['closure', 'debt-for-equity', 'resold', 'FDI', '60-percent', 'exchange,', 'cancellations', 'cancellation', '59,000', 'BHP’s']
HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is ['closure', 'debtforequity', 'resold', 'FDI', '60percent', 'exchange', 'cancellations', 'cancellation', '59000', 'BHPs']

===ROUND 1===

HOST: ['closure']
GUESSING PLAYER (HerokuOrg team) to HOST: ['closure', 'scintillation', 'perturbed', 'chloride', 'ssrt']
HOST: False
GUESSING PLAYER (GKEOrg team) to HOST: ['closure', 'closures,', 'closures', 'closure.', 'closures.']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['parcels', 'Stripping', 'C-130', 'cancellations', 'closure,']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['flotation', 'closure,', 'floater', 'closures', 'auctions']
HOST: False

===ROUND 2===

HOST: ['closure', 'debtforequity']
GUESSING PLAYER (HerokuOrg team) to HOST: ['hosted

,"Explanation for ""september"" (skipgram team)",Guess (HerokuOrg team),Guess (GKEOrg team),Guess (skipgram2 team),Guess (cbow team)
0,[closure],"[closure, scintillation, perturbed, chloride, ...","[closure, closures,, closures, closure., closu...","[parcels, Stripping, C-130, cancellations, clo...","[flotation, closure,, floater, closures, aucti..."
1,"[closure, debtforequity]","[hosted, trius, mass-media, fullest, ignoratio]","[closure, rationalization, restructuring, Re-e...","[spin-offs, recapitalizations, syndication, re...","[debt-to-capital, financings, investments, cap..."
2,"[closure, debtforequity, resold]","[electrically, mass-market, frenchies, differe...","[closure, resold, debt-for-equity, presold, op...","[non-airline, deconsolidation, divested, pass-...","[syndicates, distribution),, syndicate, deposi..."
3,"[closure, debtforequity, resold, FDI]","[neutrino, constant:, octavius, merkle, cliff-...","[resold, closure, FDI, (non-operating), ONGC,]","[non-airline, Zubsolv, depository, sale., head...","[financings, derivatives, distribution),, synd..."
4,"[closure, debtforequity, resold, FDI, 60percent]","[situation:, rhyme, blizzard, >nul, chesspieces]","[$5.07, 679,000, 2.622, ONGC,, speculative-grade]","[non-airline, w/o, contract:, orders*, sale.]","[syndicates, repositories, repository, Other-t..."
5,"[closure, debtforequity, resold, FDI, 60percen...","[cobralinks, warparound, mlv, adiposity, tauri]","[exchange, ONGC,, Levelized, Mark-to-market, d...","[w/o, contract:, orders*, Increasing, non-airl...","[syndicates, derivatives, repository, reposito..."
6,"[closure, debtforequity, resold, FDI, 60percen...","[acquires, transcript, equidistant, szi, oemed]","[cancellations, Mark-to-market, day*, (redempt...","[contract:, w/o, contracts., deferred-contract...","[syndicates, auctions,, repositories, contract..."
7,"[closure, debtforequity, resold, FDI, 60percen...","[acquires, french-man, transcript, jargon, rel...","[cancellations, cancellation, cancellations,, ...","[w/o, contracts., deferred-contract, inter-com...","[auctions,, contracts, contractions, subcontra..."
8,"[closure, debtforequity, resold, FDI, 60percen...","[morbus, maintainers, transcript, reimburse, i...","[cancellations, cancellation, cancellations,, ...","[w/o, contract:, deferred-contract, contracts....","[auctions,, contracts, syndicates, auctions, s..."
9,"[closure, debtforequity, resold, FDI, 60percen...","[>voice, sheets, ""proceedings, invitation:, bu...","[cancellations, cancellations,, cancellation, ...","[w/o, contract:, deferred-contract, contracts....","[auctions,, contracts, subcontracts, syndicate..."


HOST to EXPLAINING PLAYER: the word is "september"
EXPLAINING PLAYER to HOST: my wordlist is ['from', 'renewals', 'recapturing', 'recapture', 'receipts.', 'recorded,', 'moratorium,', 'charterers’', 'eighteen', 'of']
HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is ['from', 'renewals', 'recapturing', 'recapture', 'receipts', 'recorded', 'moratorium', 'charterers', 'eighteen', 'of']

===ROUND 1===

HOST: ['from']
GUESSING PLAYER (HerokuOrg team) to HOST: ['from', 'traslation', 'designation', 'grassroots', 'up-coming']
HOST: False
GUESSING PLAYER (GKEOrg team) to HOST: ['from', 'of', 'in', 'the', 'came']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['Inflationary', 'declared.', 'aircraft', 'sovereign', 'contract']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['lapsed', 'deported', 'revered', 'grossing', 'over']
HOST: False

===ROUND 2===

HOST: ['from', 'renewals']
GUESSING PLAYER (HerokuOrg team) to HOST: ['laymen', 'phage', 'labored', 'study~s', 'starship'

,"Explanation for ""september"" (skipgram2 team)",Guess (HerokuOrg team),Guess (GKEOrg team),Guess (skipgram team),Guess (cbow team)
0,[from],"[from, traslation, designation, grassroots, up...","[from, of, in, the, came]","[Inflationary, declared., aircraft, sovereign,...","[lapsed, deported, revered, grossing, over]"
1,"[from, renewals]","[laymen, phage, labored, study~s, starship]","[renewals, from, (redemptions), due, prepaymen...","[dividend.”, 435,000, 15,000, Excessive, overs...","[renewals, landmines, territories, corridors, ..."
2,"[from, renewals, recapturing]","[discoverd, assorted, cals, earnings, hues]","[recapturing, from, renewals, Wiping, recaptured]","[$130,000, operator's, Schuldschein, “neutral”...","[induction, deprivation, grounding, torrential..."
3,"[from, renewals, recapturing, recapture]","[volubility, dimes, holds, attitude-control, r...","[recapturing, recapture, recaptured, from, ren...","[Restrictions, Resurgens, peer, syndicated, pa...","[induction, incubation, unsatisfactory, destab..."
4,"[from, renewals, recapturing, recapture, recei...","[inspection, >universal, wide-ranging, excelle...","[recapture, recapturing, recaptured, from, str...","[TEU,, transferred, Whirlpool, Whirlpool's, sh...","[nuance, originations, induction, extinction, ..."
5,"[from, renewals, recapturing, recapture, recei...","[excellence, picolinate, >universal, prepaymen...","[recapture, recapturing, recorded, (recapture)...","[TEU,, recently-enacted, Seasonally, Whirlpool...","[syndicated, origination, percentage-of-comple..."
6,"[from, renewals, recapturing, recapture, recei...","[prior, glucoma, prepayment, dispersion, >shut...","[recapture, recapturing, recorded, (recapture)...","[TEU,, Whirlpool, transferred, Whirlpool,, sha...","[origination, originations, sale-leaseback, ag..."
7,"[from, renewals, recapturing, recapture, recei...","[*the, splinter, leveling, outlay, respective]","[charterers’, recapture, renewals, (sequential...","[TEU,, (E&C), peer, WorldCom, pre-recorded]","[originations, origination, closings,, syndica..."
8,"[from, renewals, recapturing, recapture, recei...","[labored, splinter, prior, starship, joining]","[charterers’, eighteen, (sequential, $529,000,...","[TEU,, E&Ps, twenty-six, Whirlpool, Individual]","[originations, Collateralized, origination, sy..."
9,"[from, renewals, recapturing, recapture, recei...","[splinter, labored, leveling, study~s, joining]","[eighteen, charterers’, (sequential, of, resul...","[TEU,, E&Ps, twenty-six, Individual, Whirlpool,]","[originations, Collateralized, origination, cl..."


HOST to EXPLAINING PLAYER: the word is "september"
EXPLAINING PLAYER to HOST: my wordlist is ['end-September', '(September', 'uncompleted', 'mid-September', '“December', 'end-November', 'timber', 'September', 'receding', 'completed']
HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is ['endSeptember', 'September', 'uncompleted', 'midSeptember', 'December', 'endNovember', 'timber', 'receding', 'completed']

===ROUND 1===

HOST: ['endSeptember']
GUESSING PLAYER (HerokuOrg team) to HOST: ['1989:', 'amtrak', 'summer:', 'ouster', 'varsity']
HOST: False
GUESSING PLAYER (GKEOrg team) to HOST: ['September', '(September', 'September,', "September's", 'September.']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['180-day', 'loadings', 'outnumbering', 'monthly', 'leniency']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['months', 'due', 'year', 'quarters', 'period,']
HOST: False

===ROUND 2===

HOST: ['endSeptember', 'September']
GUESSING PLAYER (HerokuOrg team) to H

,"Explanation for ""september"" (cbow team)",Guess (HerokuOrg team),Guess (GKEOrg team),Guess (skipgram team),Guess (skipgram2 team)
0,[endSeptember],"[1989:, amtrak, summer:, ouster, varsity]","[September, (September, September,, September'...","[180-day, loadings, outnumbering, monthly, len...","[months, due, year, quarters, period,]"
1,"[endSeptember, September]","[1989:, sands, corps, banquet, amtrak]","[September, December, June, November, 30,]","[leniency, (LNG), 180-day, reoffer, disbursed]","[months, September's, due, one-year, ending]"
2,"[endSeptember, September, uncompleted]","[spanish-speaking, mid-march, announced, corti...","[September, December, 2017,, June, 2016,]","[59,000, holdings, Aaa, orders, 60-percent]","[from, recapturing, recorded,, 2016-2020,, three]"
3,"[endSeptember, September, uncompleted, midSept...","[spanish-speaking, |been, cortisone, announced...","[September, December, June, 2017,, November]","[holdings, 59,000, banks, closure, 50-percent]","[from, recorded,, due, “December, (last]"
4,"[endSeptember, September, uncompleted, midSept...","[varsity, summer:, england;, londoff, tito]","[September, December, 2017,, June, 31,]","[yield-curve-control, monthly, Euribor,, manag...","[from, recorded,, November/December, “December..."
5,"[endSeptember, September, uncompleted, midSept...","[varsity, summer:, england;, amtrak, ouster]","[September, December, November, 2017,, June]","[leniency, yield-curve-control, mid-contract, ...","[from, November/December, “December, April-Dec..."
6,"[endSeptember, September, uncompleted, midSept...","[varsity, amtrak, jocks, newspaper:, scandanavia]","[September, December, November, June, 2017,]","[mid-contract, leniency, Starmine’s, yuan-deno...","[from, 10-20, shrinkage, November/December, mi..."
7,"[endSeptember, September, uncompleted, midSept...","[varsity, |been, england;, scandanavia, varco-...","[September, December, November, June, 2017,]","[loadings, monthly, Euribor,, 100,000,, 180-day]","[from, November/December, 10-20, “December, Ap..."
8,"[endSeptember, September, uncompleted, midSept...","[|been, varco-pruden, capita, england;, spanis...","[September, December, November, 2017,, June]","[Starmine’s, sandstorms, monthly, 50-percent, ...","[from, correspondents, “December, November/Dec..."


In [14]:
game.report_results(each_game=True)

=== Team scores in each game ===


,GKEOrg team,HerokuOrg team,cbow team,skipgram team,skipgram2 team
game,,,,,
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0


=== Team scores, summary ===


,explaining,guessing,total
GKEOrg team,0,0,0.0
HerokuOrg team,0,0,0.0
cbow team,0,0,0.0
skipgram team,0,0,0.0
skipgram2 team,0,0,0.0


In [15]:
PLAYERS = [
    player('skipgram team', LocalFasttextPlayer(model_skipgram)),
    player('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    player('cbow team', LocalFasttextPlayer(model_cbow))
]

N_WORDS = 10

for vocabulary_path in [
    'vocabulary/verbs_top_50.txt',
    'vocabulary/nouns_top_50.txt',
    'vocabulary/adjectives_top_50.txt'
]:
    print(vocabulary_path)
    with open(vocabulary_path) as f:
        WORDS = f.readlines()
        WORDS = [word.strip() for word in WORDS][:N_WORDS]

    game_start = pd.Timestamp.now()
    game = Game(PLAYERS, WORDS, CRITERIA, N_ROUNDS, N_EXPLAIN_WORDS, N_GUESSING_WORDS)
    game.run(verbose=False, complete=True)
    game.report_results(each_game=False)
    game_end = pd.Timestamp.now()
    display(game_end - game_start)

vocabulary/verbs_top_50.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,33,31,64.0
skipgram team,30,23,53.0
skipgram2 team,25,34,59.0


Timedelta('0 days 00:00:01.740790')

vocabulary/nouns_top_50.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,19,16,35.0
skipgram team,16,0,16.0
skipgram2 team,16,35,51.0


Timedelta('0 days 00:00:01.523222')

vocabulary/adjectives_top_50.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,44,20,64.0
skipgram team,0,20,20.0
skipgram2 team,26,30,56.0


Timedelta('0 days 00:00:01.684295')